In [1]:
!pip install opencv-python

In [2]:
import cv2
import time
import sqlite3

In [9]:
# Class to detect motion between frames
class MotionDetector:
    def __init__(self):
        self.prev_frame = None          # store previous frame
        self.last_event_time = 0        # time of last motion event

    def detect(self, frame):
        if self.prev_frame is None:
            self.prev_frame = frame
            return None

        # Calculate difference between current and previous frame
        diff = cv2.absdiff(self.prev_frame, frame)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        motion_pixels = cv2.countNonZero(thresh)
        self.prev_frame = frame

        # Only report motion if enough pixels changed and 2 seconds passed
        current_time = time.time()
        if motion_pixels > 3000 and current_time - self.last_event_time > 2:
            self.last_event_time = current_time
            return motion_pixels

        return None


In [10]:
# Create SQLite database if not exists
def create_db():
    conn = sqlite3.connect("events.db")
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS events (timestamp TEXT, event_type TEXT, value INTEGER)")
    conn.commit()
    conn.close()

# Save motion events to database
def save_event(value):
    conn = sqlite3.connect("events.db")
    cur = conn.cursor()
    cur.execute("INSERT INTO events VALUES (?, ?, ?)",
                (time.strftime("%Y-%m-%d %H:%M:%S"), "motion_detected", value))
    conn.commit()
    conn.close()


In [11]:
# Initialize database and motion detector
create_db()
detector = MotionDetector()

cap = cv2.VideoCapture(0)

frame_count = 0
max_frames = 150       # stop after 150 frames

print("Starting motion detection...")

while frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    # convert to grayscale and blur for easier motion detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    motion = detector.detect(blur)

    if motion:
        print("Motion detected:", motion)
        save_event(motion)

    frame_count += 1
    time.sleep(0.05)   # slow down loop for Jupyter

cap.release()
print("Motion detection finished")


Starting motion detection...
Motion detected: 98479
Motion detected: 166633
Motion detected: 19854
Motion detected: 101527
Motion detected: 3924
Motion detection finished


In [12]:
# Check stored motion events
conn = sqlite3.connect("events.db")
cur = conn.cursor()

cur.execute("SELECT * FROM events LIMIT 10")
rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()


('2025-12-22 16:37:37', 'motion_detected', 52044)
('2025-12-22 16:37:37', 'motion_detected', 305356)
('2025-12-22 16:37:37', 'motion_detected', 14077)
('2025-12-22 16:37:37', 'motion_detected', 29492)
('2025-12-22 16:37:38', 'motion_detected', 14921)
('2025-12-22 16:37:38', 'motion_detected', 238597)
('2025-12-22 16:37:38', 'motion_detected', 41007)
('2025-12-22 16:37:38', 'motion_detected', 8965)
('2025-12-22 16:37:38', 'motion_detected', 301002)
('2025-12-22 16:38:59', 'motion_detected', 177078)
